In [86]:
# importation of depedencies

import pandas as pd 
import numpy as np


## Loading data

In [93]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv("data/test.csv")
data_train

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,361,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,22.1,1
2186,2186,362,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,35.3,1
2187,2187,363,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,32.9,1
2188,2188,364,1022.3,16.4,15.2,13.8,14.7,92.0,93.0,0.1,40.0,18.0,1


In [15]:
data_test.isna().sum()
# We can see that there's a missing data nan



id               0
day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    1
windspeed        0
dtype: int64

In [16]:
data_test["winddirection"].mean()

np.float64(103.92318244170096)

In [7]:
data_test["winddirection"] = data_test["winddirection"].fillna(data_test["winddirection"].mean())
# Not correct way but it works

In [8]:
data_test.isna().sum()

id               0
day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
dtype: int64

### Normalisation

In [ ]:
#Non-features columns ; NF_columns  -> ['id','day',"rainfall"]
features=data_train.drop(columns=['id','day','rainfall'])
#Target ; ["rainfall"]
targets=data_train["rainfall"]

x=features.values
means=x.mean(axis=0) #mean for each columns
std=x.std(axis=0)
x_normalize= (x-means)/std

In [ ]:
data_train_n=data_train.copy()
data_train_n[features.columns]=x_normalize
data_train_n

### Imbalanced Dataset 

In [41]:
targets.value_counts()

rainfall
1    1650
0     540
Name: count, dtype: int64

In [60]:
from imblearn.over_sampling import SMOTE


#update the value of features and targets after normalize 
features=data_train_n.drop(columns=['id','day','rainfall'])
targets=data_train_n["rainfall"]



smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features.values, targets.values)


In [58]:
y_r = pd.Series(y_resampled,name="rainfall")
y_r.value_counts()

rainfall
1    1650
0    1650
Name: count, dtype: int64

In [96]:
data_train_VF= pd.DataFrame(X_resampled,columns=features.columns)
data_train_VF['rainfall']= y_resampled
data_train_VF['id']=np.arange(len(data_train_VF))+1
data_train_VF['day']=np.arange(len(data_train_VF)) % 366 +1 

#reorde columns and save it 
columns =['id','day'] + list(features.columns) +  ["rainfall"]
data_train_VF = data_train_VF[columns]
data_train_VF.to_csv("data/train_processed.csv",index=False)

In [97]:
data_train_VF

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,1,1,0.671702,-0.913809,-0.642199,-0.448815,-0.199457,0.636434,0.681269,-0.729397,-0.560901,-0.465291,1
1,2,2,1.043116,-1.798289,-1.350846,-1.259418,-0.956001,1.662224,0.847728,-1.032804,-0.685925,0.009629,1
2,3,3,1.856688,-1.232222,-1.504067,-1.496667,-2.109731,-0.902250,-1.593680,1.256536,-0.435876,-0.374349,1
3,4,4,-0.035752,-1.462187,-1.178472,-1.041939,-0.691210,1.662224,1.069675,-1.032804,-0.560901,1.393971,1
4,5,5,1.449902,-0.896120,-1.063556,-1.378043,-2.052990,-3.851394,-1.704654,-0.039837,-0.810950,0.302665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,3296,2,0.896361,-0.173770,-0.220368,-0.056342,-0.682610,-0.089462,-0.909499,-0.652118,-0.748952,0.660752,0
3296,3297,3,1.614381,-1.623131,-1.710886,-1.807024,-1.973620,-1.094424,-1.688022,1.275851,-1.061000,-0.934382,0
3297,3298,4,-0.910964,0.912511,0.916527,0.891618,0.909954,0.101534,-0.315840,1.454347,0.241032,-0.326925,0
3298,3299,5,-0.952384,0.979646,1.036035,0.931877,0.857713,-0.849226,-1.584084,1.919338,1.586145,-1.007147,0


### Dimension Reduction 

> Apply Principal Component  Analysis

In [118]:
from sklearn.decomposition import PCA

train=pd.read_csv("data/train_processed.csv",dtype=np.float32)
features=train.drop(columns=['id','day','rainfall'])
targets=train['rainfall']

pca = PCA(n_components=5,svd_solver="full") # from 10 to 5 

feat_pca=pca.fit_transform(features)
pca_df = pd.DataFrame(feat_pca, columns=["PC1", "PC2","PC3","PC4","PC5"])
pca_df["rainfall"] = targets.values

pca_df.to_csv("data/train_pca.csv",index=False)

In [117]:
pca_df

,PC1,PC2,PC3,PC4,PC5,rainfall
0,-1.681271,1.401096,-0.815937,-0.048957,0.113072,1.0
1,-3.328763,1.886631,-0.655738,-0.097774,1.136976,1.0
2,-3.002949,-2.503245,-0.580183,0.480613,0.891330,1.0
3,-2.755050,2.368489,0.803139,-0.369434,0.995535,1.0
4,-2.930984,-3.290252,0.346370,1.098036,-1.907488,0.0
...,...,...,...,...,...,...
3295,-1.315982,0.022391,0.418847,-0.649037,-0.113708,0.0
3296,-3.310956,-2.742178,-1.245059,0.196968,0.695523,0.0
3297,2.140737,0.123733,-0.013056,-0.527242,0.164404,0.0
3298,3.061349,-1.349540,-0.194924,0.679204,0.526757,0.0


### Model 